# Basic Model for Supply Chain Transportation Lanes Using the Minimum Cost Flow Optimization Model

Sabi Horvat, January 2021 - *Updated February 2021*

What does this model do?  In summary, the model chooses transportation lanes from a number of options.  In practice, the data for these options is changing at least on a quarterly basis due to the following reasons.

* Contracts with the suppliers that manufacture these products expire and require periodic renegotiation. 
* Transportation costs from third party logistics (3PLs) fluctuate depending on fuel prices and other costs, but also due to shifting demand and supply.  One example is when a truck delivers products to an area where there is not also product to load upon a return trip.  When the 3PL can't coordinate for the truck to be highly utilized, such as in this situation, the truck is being paid to "deadhead" back empty or at other times at less than a Full TruckLoad (FTL).  These cost differences account for the increase in price for the quote you receive, compared to other destinations.

Additional python formulations of Minimum Cost Flow models can be found at: 
* [COIN-OR / PuLP (github) - American Steel Problem](https://github.com/coin-or/pulp/blob/master/examples/AmericanSteelProblem.py)
* [github-alerera - MCNF Timespace Problem](https://github.com/alerera/logistics-opt-python/blob/master/pulp/mcnf_timespace.py)
* [Gurobi-github - General Landing Page for Multiple Problems ](https://gurobi.github.io/modeling-examples/)

---
## Python Model

In [1]:
import pandas as pd
from pulp import *

/Users/alberta/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Data

In [2]:
# This data has been aggregated for period of time
# The transportation lane will be effective for this projected demand, during that time

# Data
#  Suppliers at Portland, Oregon and Portland, Maine
#  Customer Demand in Chicago, Seattle, Miami, Dallas, and San Diego

locations = ['Portland,OR','Portland,ME','Chicago,IL','Seattle,WA',
            'Miami,FL','Dallas,TX','SanDiego,CA']

supply_and_demand = {'Portland,OR':[100,0],
                     'Portland,ME':[100,0],
                     'Chicago,IL' :[0,50],
                     'Seattle,WA' :[0,40],
                     'Miami,FL'   :[0,30],
                     'Dallas,TX'  :[0,50],
                     'SanDiego,CA':[0,30]}
(supply, demand) = splitDict(supply_and_demand)

route_arcs = [   ('Portland,OR','Portland,OR'),
                 ('Portland,OR','Chicago,IL'),
                 ('Portland,OR','Seattle,WA'),
                 ('Portland,OR','Miami,FL'),
                 ('Portland,OR','Dallas,TX'),
                 ('Portland,OR','SanDiego,CA'),
                 ('Portland,ME','Portland,ME'),
                 ('Portland,ME','Chicago,IL'),
                 ('Portland,ME','Seattle,WA'),
                 ('Portland,ME','Miami,FL'),
                 ('Portland,ME','Dallas,TX'),
                 ('Portland,ME','SanDiego,CA'),
                 ('Chicago,IL','Seattle,WA'),
                 ('Chicago,IL','Miami,FL'),
                 ('Chicago,IL','Dallas,TX'),
                 ('Chicago,IL','SanDiego,CA')   ]

# cost per unit (variable per unit, fixed for utilizing lane)
route_cost = {   ('Portland,OR','Portland,OR'):[0,0],
                 ('Portland,OR','Chicago,IL'): [0.5,0],
                 ('Portland,OR','Seattle,WA'): [0.1,100],
                 ('Portland,OR','Miami,FL'):   [0.9,100],
                 ('Portland,OR','Dallas,TX'):  [0.5,100],
                 ('Portland,OR','SanDiego,CA'):[0.2,100],
                 ('Portland,ME','Portland,ME'):[0,0],
                 ('Portland,ME','Chicago,IL'): [0.4,100],
                 ('Portland,ME','Seattle,WA'): [0.9,100],
                 ('Portland,ME','Miami,FL'):   [0.3,100],
                 ('Portland,ME','Dallas,TX'):  [0.6,100],
                 ('Portland,ME','SanDiego,CA'):[0.9,100],
                 ('Chicago,IL','Seattle,WA'):  [0.5,0],
                 ('Chicago,IL','Miami,FL'):    [0.2,0],
                 ('Chicago,IL','Dallas,TX'):   [0.3,0],
                 ('Chicago,IL','SanDiego,CA'): [0.5,0]   }
(variable_costs, fixed_cost) = splitDict(route_cost)

## Model

In [3]:
# Decision Variables
flow = LpVariable.dicts("Route",route_arcs,lowBound=0,upBound=100,cat="Integer")

# Model, to minimize the objective function
model = LpProblem("Minimum_Cost_Flow_Problem_Sample",LpMinimize)

# Creates the objective function, product cost is not included in this simplified model
model += lpSum([flow[a]* variable_costs[a] for a in route_arcs]), "Transportation Cost "

# Constraint: Supply the demand
for l in locations:
    model += (supply[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if j == l]) >=
             demand[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if i == l])), "Flow %s"%l


## Results

In [4]:
# Solve the model
model.solve()

# The status of the solution (Optimal, Infeasible, Unbounded, Not Solved, or Undefined)
print("Status:", LpStatus[model.status])

# The objective solution
# for v in model.variables():
#     print(v.name, "=", v.varValue)

# The optimal objective function value 
print("\nTotal Cost = ", value(model.objective))

Status: Optimal

Total Cost =  66.0


In [5]:
# Objective Solution
result_value = []
for v in model.variables():
    result_value.append(v.varValue)

result_arc = []
for v in model.variables():
    result_arc.append(v.name)

result_od = pd.DataFrame(result_arc, columns=['result_string'])
result_od

,result_string
0,"Route_('Chicago,IL',_'Dallas,TX')"
1,"Route_('Chicago,IL',_'Miami,FL')"
2,"Route_('Chicago,IL',_'SanDiego,CA')"
3,"Route_('Chicago,IL',_'Seattle,WA')"
4,"Route_('Portland,ME',_'Chicago,IL')"
5,"Route_('Portland,ME',_'Dallas,TX')"
6,"Route_('Portland,ME',_'Miami,FL')"
7,"Route_('Portland,ME',_'Portland,ME')"
8,"Route_('Portland,ME',_'SanDiego,CA')"
9,"Route_('Portland,ME',_'Seattle,WA')"


In [6]:
# Format columns "origin" and "destination"
result_od['result_string'] = result_od['result_string'].str.replace('Route_','')
result_od['origin'] = result_od['result_string'].str.split('_', expand=True)[0]
result_od['destination'] = result_od['result_string'].str.split('_', expand=True)[1]
result_od['origin'] = result_od['origin'].str.replace('[^\w\s]','', regex=True)
result_od['destination'] = result_od['destination'].str.replace('[^\w\s]','', regex=True)
result_od['flow'] = result_value
result_od = result_od.drop(['result_string'], axis=1)[result_od['flow'] > 0]
result_od

,origin,destination,flow
4,PortlandME,ChicagoIL,50.0
5,PortlandME,DallasTX,20.0
6,PortlandME,MiamiFL,30.0
11,PortlandOR,DallasTX,30.0
14,PortlandOR,SanDiegoCA,30.0
15,PortlandOR,SeattleWA,40.0


In [21]:
# Create a map with folium
import folium
center = (40,-95)
m = folium.Map(location=center, 
#                width='50%', 
#                height='60%',
               zoom_start=3, 
#                tiles='Stamen Toner'
              )

# Added tile options, as the three Stamen tiles aren't working on 3/3/24
folium.TileLayer('OpenStreetMap').add_to(m)
# folium.TileLayer('Stamen Terrain').add_to(m)
# folium.TileLayer('Stamen Toner').add_to(m)
# folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('CartoDB positron').add_to(m)
folium.TileLayer('CartoDB dark_matter').add_to(m)
folium.LayerControl().add_to(m)

nodes = pd.DataFrame({ 'Location': ['PortlandME','PortlandOR','ChicagoIL',
                                           'DallasTX','MiamiFL','SanDiegoCA','SeattleWA'],
                            'Latitude': [43.7,45.5,41.8,32.8,25.8,32.9,47.6],
                            'Longitude': [-70.3,-122.6,-87.7,-96.8,-80.2,-117.1,-122.3],
                            'Inventory': [1,1,0,0,0,0,0]
                            })
supply = folium.FeatureGroup(Inventory = 1)
demand_only = folium.FeatureGroup(Inventory = 0)
df_supply = nodes[nodes['Inventory']==1]
df_demand_only = nodes[nodes['Inventory']==0]

In [22]:
for i in range(len(df_supply)): folium.CircleMarker(
    location=[df_supply.iloc[i]['Latitude'],
              df_supply.iloc[i]['Longitude']],
    color = 'green',
    fill = True,
    fill_opacity = 0.7,
    radius=10,
    ).add_to(m)
for i in range(len(df_demand_only)): folium.CircleMarker(
    location=[df_demand_only.iloc[i]['Latitude'],
              df_demand_only.iloc[i]['Longitude']],
    color = 'blue',
    fill = True,
    fill_opacity = 0.7,
    radius=5,
    ).add_to(m)

In [23]:
# merge nodes with result_od to get origin lat long and destination lat long
add_lines = result_od.merge(nodes, left_on='origin',right_on='Location', how='inner') 
add_lines = add_lines.rename(columns={"Latitude": "Origin Latitude", "Longitude": "Origin Longitude"}).drop(['Location','Inventory'],axis=1)
add_lines = add_lines.merge(nodes, left_on='destination',right_on='Location', how='inner') 
add_lines = add_lines.rename(columns={"Latitude": "Destination Latitude", "Longitude": "Destination Longitude"}).drop(['Location','Inventory'],axis=1)
add_lines

,origin,destination,flow,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,PortlandME,ChicagoIL,50.0,43.7,-70.3,41.8,-87.7
1,PortlandME,DallasTX,20.0,43.7,-70.3,32.8,-96.8
2,PortlandOR,DallasTX,30.0,45.5,-122.6,32.8,-96.8
3,PortlandME,MiamiFL,30.0,43.7,-70.3,25.8,-80.2
4,PortlandOR,SanDiegoCA,30.0,45.5,-122.6,32.9,-117.1
5,PortlandOR,SeattleWA,40.0,45.5,-122.6,47.6,-122.3


In [24]:
# loop on rows to create O-D pairs
add_lines_df = pd.DataFrame()
for i in range(0,len(add_lines)):
    origin = pd.DataFrame((add_lines['Origin Latitude'][i],add_lines['Origin Longitude'][i]))
    destination = (add_lines['Destination Latitude'][i],add_lines['Destination Longitude'][i])
    od = [(add_lines['Origin Latitude'][i],add_lines['Origin Longitude'][i]),
         (add_lines['Destination Latitude'][i],add_lines['Destination Longitude'][i])]
    add_lines_df[i] = od
    folium.PolyLine(add_lines_df[i], color='orange').add_to(m)
m

In [25]:
m.save("folium_map.html")

In [26]:
from IPython.display import IFrame
IFrame(src='folium_map.html', width=900, height=600)